<a href="https://colab.research.google.com/github/Sudkorea/AlgorithmStudy/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4
from google.colab import drive
import json
import os

drive.mount('/content/drive')

def load_api_keys():
    key_path = '/content/drive/MyDrive/api/api_keys.json'
    if os.path.exists(key_path):
        with open(key_path, 'r') as f:
            return json.load(f)
    else:
        print("API 키 파일이 Drive에 없습니다. 샘플 키를 사용합니다.")
        return {
            "KOPIS_service_key": "sample_key_1",
            "naver_client_id": "sample_key_2",
            "naver_client_secret": "sample_key_3"
        }

api_keys = load_api_keys()
service_key = api_keys["KOPIS_service_key"]
client_id = api_keys["naver_client_id"]
client_secret = api_keys["naver_client_secret"]

Mounted at /content/drive


---

입력한 내용을 바탕으로 뮤지컬 찾아오는 코드

In [ ]:
import os
import csv
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

def search_musical_with_id(musical_id, service_key):
    URL = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{musical_id}"
    params = {
        "service": service_key
    }
    response = requests.get(URL, params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        performance = root.find("db")

        if performance is not None:
            prfnm = performance.findtext("prfnm")
            prfpdfrom = performance.findtext("prfpdfrom")
            prfpdto = performance.findtext("prfpdto")
            fcltynm = performance.findtext("fcltynm")
            genre = performance.findtext("genrenm")

            return {
                "mt20id": musical_id,
                "prfnm": prfnm,
                "fcltynm": fcltynm,
                "prfpdfrom": prfpdfrom,
                "prfpdto": prfpdto,
                "genre": genre
            }
        else:
            print(f"뮤지컬 ID {musical_id}를 찾을 수 없습니다.")
            return None
    else:
        print(f"API 요청 실패: {response.status_code}")
        return None

def process_all_musicals(service_key):
    df = pd.read_csv('/content/drive/MyDrive/data/all_musicals_code.csv')

    # 새로운 컬럼을 추가합니다.
    df['prfnm'] = ''
    df['fcltynm'] = ''
    df['prfpdfrom'] = ''
    df['prfpdto'] = ''
    df['genre'] = ''

    # tqdm을 사용하여 진행 상황을 표시합니다.
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing musicals"):
        musical_id = row['mt20id']
        musical_info = search_musical_with_id(musical_id, service_key)

        if musical_info:
            df.at[index, 'prfnm'] = musical_info['prfnm']
            df.at[index, 'fcltynm'] = musical_info['fcltynm']
            df.at[index, 'prfpdfrom'] = musical_info['prfpdfrom']
            df.at[index, 'prfpdto'] = musical_info['prfpdto']
            df.at[index, 'genre'] = musical_info['genre']

    return df

# 메인 실행 코드
if __name__ == "__main__":
    all_musicals = process_all_musicals(service_key)

    # 결과를 DataFrame으로 변환
    df = pd.DataFrame(all_musicals)

    # CSV 파일로 저장
    output_file = "/content/drive/MyDrive/data/all_musicals.csv"
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"모든 뮤지컬 정보가 {output_file}에 저장되었습니다.")

Processing musicals: 100%|██████████| 1128/1128 [08:16<00:00,  2.27it/s]

모든 뮤지컬 정보가 /content/drive/MyDrive/data/all_low_musicals.csv에 저장되었습니다.


---

시작점 다시 잡아주는 코드

In [ ]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm

ACCUMULATED_FILE_PATH = '/content/drive/MyDrive/data/all_musicals.csv'
EXTRACTED_CSV_DIR = '/content/drive/MyDrive/data/musical'


def update_start_dates():
    """
    모든 뮤지컬의 start_date를 업데이트하는 함수
    """
    # 누적된 데이터 로드
    accumulated_df = pd.read_csv(ACCUMULATED_FILE_PATH)

    updated_count = 0

    # tqdm을 사용하여 진행 상황 표시
    for index, row in tqdm(accumulated_df.iterrows(), total=accumulated_df.shape[0], desc="Updating start dates"):
        performance_code = row['mt20id']
        csv_file_path = os.path.join(EXTRACTED_CSV_DIR, f'{performance_code}.csv')

        if not os.path.exists(csv_file_path):
            print(f"Warning: CSV file not found for performance code {performance_code}")
            continue

        try:
            # CSV 파일에서 첫 번째 행만 읽기
            extracted_df = pd.read_csv(csv_file_path, nrows=1)

            if extracted_df.empty:
                print(f"Warning: Extracted CSV file is empty for performance code {performance_code}")
                continue

            # '예매일시'에서 날짜 부분만 추출
            new_start_date = extracted_df['예매일시'].iloc[0].split()[0]

            # 날짜 형식을 YYYY.MM.dd로 변경
            new_start_date = datetime.strptime(new_start_date, '%Y-%m-%d').strftime('%Y.%m.%d')

            # 기존 start_date의 형식을 확인하고 처리
            old_start_date = row['prfpdfrom']
            if '-' in old_start_date:
                old_start_date = datetime.strptime(old_start_date, '%Y-%m-%d').strftime('%Y.%m.%d')

            # prfpdfrom 업데이트가 필요한 경우에만 업데이트
            if old_start_date != new_start_date:
                accumulated_df.at[index, 'prfpdfrom'] = new_start_date
                updated_count += 1
                print(f"Start date updated for performance code {performance_code}: {old_start_date} -> {new_start_date}")

        except Exception as e:
            print(f"Error processing performance code {performance_code}: {str(e)}")

    # 변경사항 저장
    accumulated_df.to_csv(ACCUMULATED_FILE_PATH, index=False, encoding='utf-8-sig')
    print(f"Total updates: {updated_count}")

# 함수 실행
update_start_dates()

Updating start dates:   0%|          | 1/1128 [00:04<1:26:39,  4.61s/it]

Start date updated for performance code PF185428: 2022.01.21 -> 2022.12.25


Updating start dates:   0%|          | 3/1128 [00:05<24:57,  1.33s/it]

Start date updated for performance code PF196014: 2022.09.17 -> 2022.08.10


Updating start dates:   0%|          | 4/1128 [00:06<21:07,  1.13s/it]

...

Start date updated for performance code PF190876: 2022.07.02 -> 2022.05.10


Updating start dates: 100%|█████████▉| 1127/1128 [05:56<00:00,  3.74it/s]

Start date updated for performance code PF188084: 2022.04.15 -> 2022.03.15


Updating start dates: 100%|██████████| 1128/1128 [05:57<00:00,  3.16it/s]

Start date updated for performance code PF223309: 2023.10.07 -> 2023.08.02
Total updates: 1111


---

기사 크롤링하는 코드

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import pandas as pd
from tqdm import tqdm

def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

def get_full_article_content(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        soup = BeautifulSoup(response.text, 'html.parser')
        article_body = soup.find('article', id='dic_area')
        journalist = soup.select_one('em.media_end_head_journalist_name')
        journalist = journalist.text.strip() if journalist else ''
        if article_body:
            paragraphs = article_body.find_all('p')
            return [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)], journalist
    except Exception as e:
        print(f"An error occurred while processing URL {url}: {str(e)}")
    return [], ''

def search_news(query, start_date, end_date, client_id, client_secret):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    articles = []
    start = 1
    display = 100

    while True:
        params = {
            "query": query,
            "sort": "date",
            "start": start,
            "display": display,
            "start_date": start_date.strftime("%Y%m%d"),
            "end_date": end_date.strftime("%Y%m%d")
        }

        response = requests.get(url, headers=headers, params=params)
        result = json.loads(response.text)

        print(f"API Response for query '{query}': {result.get('total', 0)} total results")

        if "items" in result:
            articles.extend(result["items"])
            print(f"Retrieved {len(result['items'])} articles in this request")

            if len(result["items"]) < display:
                break

            start += display
        else:
            print(f"No items found in the response. Response: {result}")
            break

    return articles

def calculate_article_score(article, musical_name, theater_name):
    content = ' '.join(article['full_content']).lower()
    title = article['title'].lower()
    musical_name = musical_name.lower()
    theater_name = theater_name.lower()

    score = 0

    if musical_name in title:
        score += 10

    keywords = {
        musical_name: 5,
        theater_name: 3,
        "뮤지컬": 4,
        "공연": 3,
        "티켓": 2,
        "예매": 2,
        "배우": 1,
        "출연": 1,
        "관람": 1,
        "무대": 1,
        "연출": 1,
    }

    for keyword, points in keywords.items():
        score += content.count(keyword) * points

    sentences = re.split(r'[.!?]+', content)
    for i, sentence in enumerate(sentences):
        if musical_name in sentence and "뮤지컬" in sentence:
            score += 5
        if i > 0 and musical_name in sentence and "뮤지컬" in sentences[i-1]:
            score += 3

    return score

def get_musical_news(musical_name, theater_name, start_date, end_date, client_id, client_secret):
    query = f'"{musical_name}" "{theater_name}"'
    print(f"\nSearching for: {query}")
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

    articles = search_news(query, start_date, end_date, client_id, client_secret)

    filtered_articles = []
    for i, article in enumerate(articles):
        print(f"\nChecking article {i+1}: {article['link']}")
        article['full_content'], article['journalist'] = get_full_article_content(article['link'])

        score = calculate_article_score(article, musical_name, theater_name)
        article['score'] = score
        print(f"Article {i+1} score: {score}")

        if score >= 10:
            filtered_articles.append(article)
            print(f"Article {i+1}: Added to filtered list")
        else:
            print(f"Article {i+1}: Filtered out")

    filtered_articles.sort(key=lambda x: x['score'], reverse=True)
    print(f"Total filtered articles: {len(filtered_articles)}")
    return filtered_articles

def save_to_csv(articles, file_path):
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Article Title', 'Publication Date', 'Article URL', 'Score', 'Journalist'] +
                        [f'Paragraph {i}' for i in range(1, 11)])

        for article in articles:
            row = [
                remove_html_tags(article['title']),
                article['pubDate'],
                article['link'],
                article['score'],
                article['journalist']
            ]
            for paragraph in article['full_content'][:10]:
                row.append(paragraph)
            writer.writerow(row)

def process_musical(musical_name, theater_name, start_date, end_date, performance_code, client_id, client_secret):
    start_date = datetime.strptime(start_date, "%Y.%m.%d")
    end_date = datetime.strptime(end_date, "%Y.%m.%d")

    print(f"\nProcessing: {musical_name} at {theater_name}")
    articles = get_musical_news(musical_name, theater_name, start_date, end_date, client_id, client_secret)

    file_path = f"/content/drive/MyDrive/data/news/{performance_code}.csv"
    save_to_csv(articles, file_path)

    print(f"Processed: {musical_name}, Found articles: {len(articles)}")
    print(f"Results saved to: {file_path}")

def process_all_musicals(musicals_df, client_id, client_secret):
    for _, musical in tqdm(musicals_df.iterrows(), total=len(musicals_df)):
        if _ < 39:
            continue
        musical_name = musical['prfnm']
        theater_name = musical['fcltynm']
        start_date = musical['prfpdfrom']
        end_date = musical['prfpdto']
        performance_code = musical['mt20id']
        try:
            articles = process_musical(musical_name, theater_name, start_date, end_date, performance_code, client_id, client_secret)
            if articles:
                file_path = f"/content/drive/MyDrive/data/news/{musical['mt20id']}.csv"
                save_to_csv(articles, file_path)
                print(f"Saved {len(articles)} articles for {musical['prfnm']}")
            else:
                print(f"No relevant articles found for {musical['prfnm']}")
        except Exception as e:
            print(f"Error processing {musical['prfnm']}: {str(e)}")

if __name__ == "__main__":
    musicals_df = pd.read_csv("/content/drive/MyDrive/data/all_musicals.csv")
    process_all_musicals(musicals_df, client_id, client_secret)

  0%|          | 0/82 [00:00<?, ?it/s]


Processing: 셰익스피어 인 러브 at 예술의전당

Searching for: "셰익스피어 인 러브" "예술의전당"
Date range: 2022-12-16 to 2023-03-26
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 total results
Retrieved 100 articles in this request
API Response for query '"셰익스피어 인 러브" "예술의전당"': 1229 t

 48%|████▊     | 39/82 [12:24<13:40, 19.08s/it]

Article 671 score: 10
Article 671: Added to filtered list

Checking article 672: https://m.entertain.naver.com/article/011/0004153446


KeyboardInterrupt: 

---

기사 수 찾는 코드

In [ ]:
import pandas as pd
from datetime import datetime, timezone
import os

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        return datetime.strptime(date_str, '%Y.%m.%d')

def process_musical_article_dates(performance_code, start_date, end_date):
    input_file_path = f"/content/drive/MyDrive/data/news/{performance_code}.csv"
    output_file_path = f"/content/drive/MyDrive/data/news/article_date/{performance_code}_article_date.csv"

    if not os.path.exists(input_file_path):
        print(f"File not found: {input_file_path}")
        return 0

    try:
        data = pd.read_csv(input_file_path)

        if 'Date' not in data.columns:
            print(f"'Date' column not found in {input_file_path}")
            return 0

        data['ParsedDate'] = pd.to_datetime(data['Date'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')

        # Check if any dates failed to parse
        if data['ParsedDate'].isnull().any():
            print(f"Some dates failed to parse in {input_file_path}")
            return 0

        start_date = pd.to_datetime(start_date).tz_localize('UTC')
        end_date = pd.to_datetime(end_date).tz_localize('UTC')

        mask = (data['ParsedDate'] >= start_date) & (data['ParsedDate'] <= end_date)
        filtered_df = data.loc[mask]

        date_counts = filtered_df['ParsedDate'].dt.date.value_counts().sort_index()
        dates_with_3_plus_articles = date_counts[date_counts >= 3]

        # Prepare the output DataFrame
        output_df = pd.DataFrame({
            'Date': dates_with_3_plus_articles.index,
            'ArticleCount': dates_with_3_plus_articles.values
        })

        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

        # Save the output DataFrame
        output_df.to_csv(output_file_path, index=False)

        print(f"Article dates saved to {output_file_path}")
        return len(filtered_df)
    except Exception as e:
        print(f"Error processing {input_file_path}: {str(e)}")
        return 0

def process_musicals():
    all_musicals_path = "/content/drive/MyDrive/data/all_musicals.csv"
    output_path = "/content/drive/MyDrive/data/all_musicals_updated.csv"

    all_musicals = pd.read_csv(all_musicals_path)

    all_musicals['start_date'] = all_musicals['prfpdfrom'].apply(parse_date)
    all_musicals['end_date'] = all_musicals['prfpdto'].apply(parse_date)

    all_musicals['article_count'] = all_musicals.apply(
        lambda row: process_musical_article_dates(row['mt20id'], row['start_date'], row['end_date']),
        axis=1
    )

    # Filter musicals with at least 30 articles
    filtered_musicals = all_musicals[all_musicals['article_count'] >= 30].copy()

    # Categorize musicals based on article count
    def categorize_count(count):
        if 30 <= count <= 49:
            return 0
        elif 50 <= count <= 69:
            return 1
        elif 70 <= count <= 99:
            return 2
        else:
            return 3

    filtered_musicals['group'] = filtered_musicals['article_count'].apply(categorize_count)

    # Save the updated data
    filtered_musicals.to_csv(output_path, index=False)

    print(f"Updated data saved to {output_path}")
    print(f"Total musicals processed: {len(all_musicals)}")
    print(f"Musicals with 30+ articles: {len(filtered_musicals)}")
    print("\nGroup distribution:")
    print(filtered_musicals['group'].value_counts().sort_index())

if __name__ == "__main__":
    process_musicals()

'Date' column not found in /content/drive/MyDrive/data/news/PF209957.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF209933_article_date.csv
'Date' column not found in /content/drive/MyDrive/data/news/PF209865.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF209863_article_date.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF209861_article_date.csv
'Date' column not found in /content/drive/MyDrive/data/news/PF209894.csv
'Date' column not found in /content/drive/MyDrive/data/news/PF177100.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF176020_article_date.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF175711_article_date.csv
'Date' column not found in /content/drive/MyDrive/data/news/PF175093.csv
Article dates saved to /content/drive/MyDrive/data/news/article_date/PF174940_article_date.csv
'Date' column not found in /content/drive/MyDrive/data/news/PF173

-----

기사와 별개로, 어느 요일에 어떤 날 티켓을 가장 많이 샀는지 heatmap을 보여주는 코드.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def advanced_booking_analysis(performance_code):
    # Load data
    ticket_data = pd.read_csv(f'/content/drive/MyDrive/data/musical/{performance_code}.csv')

    # Convert date columns to datetime
    for col in ['공연일시', '예매일시']:
        ticket_data[col] = pd.to_datetime(ticket_data[col], errors='coerce')

    # Remove rows with missing values
    ticket_data = ticket_data.dropna(subset=['공연일시', '예매일시', '예매수'])

    # Add day of week information
    ticket_data['performance_day'] = ticket_data['공연일시'].dt.dayofweek
    ticket_data['booking_day'] = ticket_data['예매일시'].dt.dayofweek

    # Day name mapping
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    ticket_data['performance_day'] = ticket_data['performance_day'].map(lambda x: day_names[x])
    ticket_data['booking_day'] = ticket_data['booking_day'].map(lambda x: day_names[x])

    # Create 7x7 grid data (sum)
    grid_sum = pd.pivot_table(ticket_data, values='예매수', index='booking_day',
                              columns='performance_day', aggfunc='sum', fill_value=0)

    # Create 7x7 grid data (mean)
    #grid_mean = pd.pivot_table(ticket_data, values='예매수', index='booking_day',
    #                           columns='performance_day', aggfunc='mean', fill_value=0)

    # Ensure correct order of days
    grid_sum = grid_sum.reindex(index=day_names, columns=day_names)
    #grid_mean = grid_mean.reindex(index=day_names, columns=day_names)

    # Visualization
    #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    fig, ax1 = plt.subplots(1, 1, figsize=(20, 8))
    # Heatmap for sum
    sns.heatmap(grid_sum, annot=True, fmt='g', cmap='YlOrRd', ax=ax1)
    ax1.set_title(f'Total Ticket Bookings (Booking Day vs Performance Day) - {performance_code}')
    ax1.set_xlabel('Performance Day')
    ax1.set_ylabel('Booking Day')

    # Heatmap for mean
#    sns.heatmap(grid_mean, annot=True, fmt='.2f', cmap='YlOrRd', ax=ax2)
#    ax2.set_title(f'Average Ticket Bookings (Booking Day vs Performance Day) - {performance_code}')
#    ax2.set_xlabel('Performance Day')
#    ax2.set_ylabel('Booking Day')

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print("Summary of Total Bookings:")
    print(grid_sum.sum().sort_values(ascending=False))
#    print("\nSummary of Average Bookings:")
#    print(grid_mean.mean().sort_values(ascending=False))

# Main execution code
musicals = pd.read_csv('/content/drive/MyDrive/data/all_musicals_updated.csv')

for idx, musical in musicals.iterrows():
    print(f"\nAdvanced analysis for musical: {musical['mt20id']}")
    advanced_booking_analysis(musical['mt20id'])
    print("\n" + "="*50)

---

예매 데이터와 기사 날짜를 긁어와서, 보도자료 전후 각 1주일동안 예매 추이 분석하여 보도자료가 효과가 있는지를 확인해주는 함수.
x축이 너무 못생기게 나와서 로그스케일로 변경함.

In [ ]:
#IRQ
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy.stats import ttest_ind, norm

def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

def load_and_preprocess_data(performance_code):
    # Load data
    ticket_data = pd.read_csv(f'/content/drive/MyDrive/data/musical/{performance_code}.csv')
    press_releases = pd.read_csv(f"/content/drive/MyDrive/data/news/article_date/{performance_code}_article_date.csv")

    # Convert date columns to datetime
    ticket_data['예매일시'] = pd.to_datetime(ticket_data['예매일시'])
    press_releases['Date'] = pd.to_datetime(press_releases['Date'])

    # Preprocess data
    ticket_data['할인금액'] = pd.to_numeric(ticket_data['할인금액'], errors='coerce')
    ticket_data['예매금액'] = pd.to_numeric(ticket_data['예매금액'], errors='coerce')
    ticket_data['연령'] = pd.to_numeric(ticket_data['연령'], errors='coerce')
    ticket_data['연령'] = 2023 - ticket_data['연령']  # Convert birth year to age
    ticket_data['판매좌석수'] = pd.to_numeric(ticket_data['판매좌석수'].str.replace('석', ''), errors='coerce')

    # Calculate discount rate
    ticket_data['원금액'] = ticket_data['예매금액'] + ticket_data['할인금액']
    ticket_data['할인율'] = ticket_data['할인금액'] / ticket_data['원금액']

    # Add seasonality features
    ticket_data['요일'] = ticket_data['예매일시'].dt.dayofweek
    ticket_data['월'] = ticket_data['예매일시'].dt.month

    # Remove invalid data
    ticket_data = ticket_data[(ticket_data['연령'] >= 0) & (ticket_data['연령'] <= 100)]
    ticket_data = ticket_data[ticket_data['예매금액'] > 0]
    ticket_data = ticket_data[ticket_data['판매좌석수'] > 0]

    # Remove outliers using IQR method
    columns_to_check = ['예매금액', '할인금액', '판매좌석수', '할인율']
    ticket_data = remove_outliers_iqr(ticket_data, columns_to_check)

    return ticket_data, press_releases

def meta_analysis(effect_sizes, standard_errors):
    weights = 1 / (standard_errors ** 2)
    weighted_mean = np.sum(effect_sizes * weights) / np.sum(weights)
    weighted_var = 1 / np.sum(weights)
    z_score = weighted_mean / np.sqrt(weighted_var)
    p_value = 2 * (1 - norm.cdf(abs(z_score)))
    return {'effect_size': weighted_mean, 'p_value': p_value}

def analyze_press_release_impact_advanced(performance_code, performance_name):
    try:
        ticket_data, press_releases = load_and_preprocess_data(performance_code)

        # Group ticket data by date
        daily_data = ticket_data.groupby(ticket_data['예매일시'].dt.date).agg({
            '예매수': 'sum',
            '할인금액': 'mean',
            '예매금액': 'mean',
            '연령': 'mean',
            '판매좌석수': 'mean',
            '요일': 'first',
            '월': 'first',
            '할인율': 'mean'
        }).reset_index()
        daily_data['예매일시'] = pd.to_datetime(daily_data['예매일시'])

        # Create binary variable for press release periods
        daily_data['press_release_period'] = 0
        for date in press_releases['Date']:
            mask = (daily_data['예매일시'] >= date - pd.Timedelta(days=7)) & (daily_data['예매일시'] <= date + pd.Timedelta(days=7))
            daily_data.loc[mask, 'press_release_period'] = 1

        # Add age group variables
        daily_data['age_under40'] = (ticket_data['연령'] < 40).mean()
        daily_data['age_40plus'] = (ticket_data['연령'] >= 40).mean()

        # Prepare data for regression
        X = daily_data[['press_release_period', '할인율', '예매금액', '연령', '판매좌석수', '요일', '월', 'age_under40', 'age_40plus']]
        y = daily_data['예매수']

        # Linear Regression
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()

        # Calculate average effects
        avg_effects = {
            'press_release': daily_data[daily_data['press_release_period'] == 1]['예매수'].mean() - daily_data[daily_data['press_release_period'] == 0]['예매수'].mean(),
            'discount_rate': model.params['할인율'],
            'ticket_price': model.params['예매금액'],
            'age': model.params['연령'],
            'seats_sold': model.params['판매좌석수'],
            'age_under40': model.params['age_under40'],
            'age_40plus': model.params['age_40plus']
        }

        # Perform statistical tests
        pr_period = daily_data[daily_data['press_release_period'] == 1]['예매수']
        non_pr_period = daily_data[daily_data['press_release_period'] == 0]['예매수']
        t_stat, p_value = ttest_ind(pr_period, non_pr_period)

        return {
            'performance_id': performance_code,
            'performance_name': performance_name,
            'avg_effects': avg_effects,
            't_statistic': t_stat,
            'p_value': p_value,
            'press_release_coef': model.params['press_release_period'],
            'press_release_pvalue': model.pvalues['press_release_period'],
            'discount_rate_coef': model.params['할인율'],
            'discount_rate_pvalue': model.pvalues['할인율']
        }

    except Exception as e:
        return {
            'performance_id': performance_code,
            'performance_name': performance_name,
            'error': str(e)
        }

# Analyze all performances
musicals = pd.read_csv('/content/drive/MyDrive/data/all_musicals_updated.csv')
performance_codes = musicals['mt20id']
results = []

for code in performance_codes:
    result = analyze_press_release_impact_advanced(code, '')  # performance name is empty for brevity
    if 'error' not in result:  # Only add results without errors
        results.append(result)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Calculate overall effect
press_release_effects = df_results['avg_effects'].apply(lambda x: x['press_release']).dropna()
overall_effect = press_release_effects.mean()

# Perform t-test only on non-nan values
overall_ttest = ttest_ind(press_release_effects, np.zeros(len(press_release_effects)))

# Perform meta-analysis
effect_sizes = df_results['press_release_coef'].dropna()
standard_errors = df_results['press_release_coef'].dropna() / df_results['t_statistic'].dropna()
meta_result = meta_analysis(effect_sizes, standard_errors)

# Visualization
plt.figure(figsize=(12, 6))
sns.histplot(df_results['press_release_coef'].dropna(), kde=True)
plt.axvline(x=0, color='r', linestyle='--')
plt.title('Distribution of Press Release Effects Across Performances')
plt.xlabel('Effect Size')
plt.ylabel('Count')
plt.savefig('press_release_effects_distribution.png')
plt.close()

print(f"Overall average effect of press releases: {overall_effect}")
print(f"Overall t-test result: t-statistic = {overall_ttest.statistic}, p-value = {overall_ttest.pvalue}")
print(f"Meta-analysis result: effect size = {meta_result['effect_size']}, p-value = {meta_result['p_value']}")
print("\nEffect of discount rate:")
print(df_results[['performance_id', 'discount_rate_coef', 'discount_rate_pvalue']])

# Save results
df_results.to_csv('/content/drive/MyDrive/data/press_release_analysis_results.csv', index=False)

Overall average effect of press releases: 203.37185181649932
Overall t-test result: t-statistic = 2.5147409539620313, p-value = 0.014025012644083129
Meta-analysis result: effect size = 15.844850684065978, p-value = 0.0025334506108962707

Effect of discount rate:
   performance_id  discount_rate_coef  discount_rate_pvalue
0        PF189859       -4.062244e-09          4.636420e-01
1        PF199307       -2.978502e-08          1.446974e-03
2        PF213616        7.924460e-08          2.501618e-01
3        PF218472        6.260948e-09          3.910421e-01
4        PF198434        1.497378e-08          1.878013e-01
5        PF221095       -4.765663e-06          3.382730e-01
6        PF186531        1.284231e-08          1.883656e-01
7        PF191728       -1.630166e-08          2.009229e-02
8        PF191916        5.514207e-09          2.965903e-01
9        PF194642       -1.507900e-09          6.998696e-01
10       PF200830        2.623842e-07          9.127841e-01
11       PF186855

로그 스케일에 오류가 있어서, 너무 튀는 값을 제거하고 로그를 제거한 함수

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy.stats import ttest_ind, norm

def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

def load_and_preprocess_data(performance_code):
    # Load data
    ticket_data = pd.read_csv(f'/content/drive/MyDrive/data/musical/{performance_code}.csv')
    press_releases = pd.read_csv(f"/content/drive/MyDrive/data/news/article_date/{performance_code}_article_date.csv")

    # Convert date columns to datetime
    ticket_data['예매일시'] = pd.to_datetime(ticket_data['예매일시'])
    press_releases['Date'] = pd.to_datetime(press_releases['Date'])

    # Preprocess data
    ticket_data['할인금액'] = pd.to_numeric(ticket_data['할인금액'], errors='coerce')
    ticket_data['예매금액'] = pd.to_numeric(ticket_data['예매금액'], errors='coerce')
    ticket_data['연령'] = pd.to_numeric(ticket_data['연령'], errors='coerce')
    ticket_data['연령'] = 2023 - ticket_data['연령']  # Convert birth year to age
    ticket_data['판매좌석수'] = pd.to_numeric(ticket_data['판매좌석수'].str.replace('석', ''), errors='coerce')

    # Calculate discount rate
    ticket_data['원금액'] = ticket_data['예매금액'] + ticket_data['할인금액']
    ticket_data['할인율'] = ticket_data['할인금액'] / ticket_data['원금액']

    # Add seasonality features
    ticket_data['요일'] = ticket_data['예매일시'].dt.dayofweek
    ticket_data['월'] = ticket_data['예매일시'].dt.month

    # Remove invalid data
    ticket_data = ticket_data[(ticket_data['연령'] >= 0) & (ticket_data['연령'] <= 100)]
    ticket_data = ticket_data[ticket_data['예매금액'] > 0]
    ticket_data = ticket_data[ticket_data['판매좌석수'] > 0]

    # Remove outliers using IQR method
    columns_to_check = ['예매금액', '할인금액', '판매좌석수', '할인율']
    ticket_data = remove_outliers_iqr(ticket_data, columns_to_check)

    return ticket_data, press_releases


def meta_analysis(effect_sizes, standard_errors):
    weights = 1 / (standard_errors ** 2)
    weighted_mean = np.sum(effect_sizes * weights) / np.sum(weights)
    weighted_var = 1 / np.sum(weights)
    z_score = weighted_mean / np.sqrt(weighted_var)
    p_value = 2 * (1 - norm.cdf(abs(z_score)))
    return {'effect_size': weighted_mean, 'p_value': p_value}



def analyze_press_release_impact_advanced(performance_code, performance_name):
    try:
        ticket_data, press_releases = load_and_preprocess_data(performance_code)

        # Group ticket data by date
        daily_data = ticket_data.groupby(ticket_data['예매일시'].dt.date).agg({
            '예매수': 'sum',
            '할인금액': 'mean',
            '예매금액': 'mean',
            '연령': 'mean',
            '판매좌석수': 'mean',
            '요일': 'first',
            '월': 'first',
            '할인율': 'mean'
        }).reset_index()
        daily_data['예매일시'] = pd.to_datetime(daily_data['예매일시'])

        # Create binary variable for press release periods
        daily_data['press_release_period'] = 0
        for date in press_releases['Date']:
            mask = (daily_data['예매일시'] >= date - pd.Timedelta(days=7)) & (daily_data['예매일시'] <= date + pd.Timedelta(days=7))
            daily_data.loc[mask, 'press_release_period'] = 1

        # Add age group variables
        daily_data['age_under40'] = (ticket_data['연령'] < 40).mean()
        daily_data['age_40plus'] = (ticket_data['연령'] >= 40).mean()

        # Prepare data for regression
        X = daily_data[['press_release_period', '할인율', '예매금액', '연령', '판매좌석수', '요일', '월', 'age_under40', 'age_40plus']]
        y = daily_data['예매수']

        # Linear Regression
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()

        # Calculate average effects
        avg_effects = {
            'press_release': daily_data[daily_data['press_release_period'] == 1]['예매수'].mean() - daily_data[daily_data['press_release_period'] == 0]['예매수'].mean(),
            'discount_rate': model.params['할인율'],
            'ticket_price': model.params['예매금액'],
            'age': model.params['연령'],
            'seats_sold': model.params['판매좌석수'],
            'age_under40': model.params['age_under40'],
            'age_40plus': model.params['age_40plus']
        }

        # Perform statistical tests
        pr_period = daily_data[daily_data['press_release_period'] == 1]['예매수']
        non_pr_period = daily_data[daily_data['press_release_period'] == 0]['예매수']
        t_stat, p_value = ttest_ind(pr_period, non_pr_period)

        return {
            'performance_id': performance_code,
            'performance_name': performance_name,
            'avg_effects': avg_effects,
            't_statistic': t_stat,
            'p_value': p_value,
            'press_release_coef': model.params['press_release_period'],
            'press_release_pvalue': model.pvalues['press_release_period'],
            'discount_rate_coef': model.params['할인율'],
            'discount_rate_pvalue': model.pvalues['할인율']
        }

    except Exception as e:
        return {
            'performance_id': performance_code,
            'performance_name': performance_name,
            'error': str(e)
        }


# Analyze all performances
musicals = pd.read_csv('/content/drive/MyDrive/data/all_musicals_updated.csv')
performance_codes = musicals['mt20id']
results = []

for code in performance_codes:
    result = analyze_press_release_impact_advanced(code, '')  # performance name is empty for brevity
    if 'error' not in result:  # Only add results without errors
        results.append(result)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Calculate overall effect
press_release_effects = df_results['avg_effects'].apply(lambda x: x['press_release']).dropna()
overall_effect = press_release_effects.mean()

# Perform t-test only on non-nan values
overall_ttest = ttest_ind(press_release_effects, np.zeros(len(press_release_effects)))

# 기존 코드에서 effect_sizes 계산
effect_sizes = df_results['press_release_coef'].dropna()

# 극단값 제거 (예: 99퍼센타일 이상의 값 제거)
upper_limit = np.percentile(effect_sizes, 99)
effect_sizes_filtered = effect_sizes[effect_sizes <= upper_limit]

# 그래프 생성
plt.figure(figsize=(12, 6))

# 히스토그램 생성
sns.histplot(effect_sizes_filtered, kde=True, bins=50)

# 수직선 추가 (효과 없음 = 0)
plt.axvline(x=0, color='r', linestyle='--', label='No effect (x=0)')

# 그래프 커스터마이징
plt.title('Distribution of Press Release Effects on Ticket Sales', fontsize=16)
plt.xlabel('Effect Size (Change in Number of Tickets Sold)', fontsize=14)
plt.ylabel('Count of Performances', fontsize=14)

# x축 범위 설정 (데이터의 최소값과 99퍼센타일 값 사이)
x_min, x_max = effect_sizes_filtered.min(), effect_sizes_filtered.max()
plt.xlim(x_min, x_max)

# x축 눈금 설정 (더 세밀하게)
plt.xticks(np.linspace(x_min, x_max, 15), rotation=45)

# 범례 추가
plt.legend()

# 그리드 추가
plt.grid(True, which="both", ls="-", alpha=0.2)
plt.tight_layout()

# 그래프 저장
plt.savefig('press_release_effects_distribution_detailed.png', dpi=300, bbox_inches='tight')
plt.close()